In [1]:
import pandas as pd
import numpy as np

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import gensim

In [2]:
#load dataset and preprocess
twt = pd.read_csv('train.csv')
twt = twt.set_index('id')

In [3]:
print(twt.info())
twt.loc[~twt['keyword'].isna()]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 1 to 10873
Data columns (total 4 columns):
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(1), object(3)
memory usage: 297.4+ KB
None


,keyword,location,text,target
id,,,,
48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...
10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0


## 0. Data Preprocessing

### 0.1 Data Cleaning

In [4]:
stop_words = stopwords.words('english')
porter = PorterStemmer()

In [5]:
def processing (sentence):
    result = sentence.lower() #Lower case 
    result = re.sub(r'\d+', '', result) #Removing numbers
    result = result.translate(str.maketrans('', '', string.punctuation)) #Remove weird characters
    result = result.strip() #Eliminate blanks from begining and end of setences
    result = result.split() #Separate into words
    result = [w for w in result if not w in stop_words] #Eliminate stop_words
    result = [porter.stem(word) for word in result] #Stem Words
    return (result)

In [6]:
text = list(twt["text"])
processing(text[0])

['deed', 'reason', 'earthquak', 'may', 'allah', 'forgiv', 'us']

### 0.2 Embeding

In [7]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [25]:
def tweet_vec (tweet, word2vec):
    word_vecs = [word2vec.get_vector(w) for w in tweet if w in word2vec.vocab]
#     print(tweet)
#     print('Number of words: {}'.format(len(word_vecs)))
    if len(word_vecs) > 1:
        return np.stack(word_vecs).mean(0)
    else:
        return None

In [9]:
prep_text = [processing(i) for i in text]

In [26]:
final_data = []

# prep_text[0]
# tweet_vec(prep_text[0], word2vec)
for i in prep_text:
    vec = tweet_vec(i,word2vec)
    if vec is not None:    
        final_data.append(vec)